# Information Theory: Perplexed by Texts

**In this exercise we will calculate some information-theoretic quantities using the text of classic books from the Gutenberg project.**

**The following code uses NLTK (one of the standard NLP libraries) to load the text of all of the books in an archive of the Gutenberg project:**

In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import re
import warnings
warnings.simplefilter(action='ignore')

[nltk_data] Downloading package gutenberg to /home/gal/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
text = re.sub(r'[^A-Za-z0-9\.,!\?\'" ]', '_', gutenberg.raw())

**We see that this text starts from "Emma" by Jane Austen. We applied some character normalization so that the set of characters used will be limited:**

In [3]:
print(text[0:1000])

_Emma by Jane Austen 1816___VOLUME I__CHAPTER I___Emma Woodhouse, handsome, clever, and rich, with a comfortable home_and happy disposition, seemed to unite some of the best blessings_of existence_ and had lived nearly twenty_one years in the world_with very little to distress or vex her.__She was the youngest of the two daughters of a most affectionate,_indulgent father_ and had, in consequence of her sister's marriage,_been mistress of his house from a very early period.  Her mother_had died too long ago for her to have more than an indistinct_remembrance of her caresses_ and her place had been supplied_by an excellent woman as governess, who had fallen little short_of a mother in affection.__Sixteen years had Miss Taylor been in Mr. Woodhouse's family,_less as a governess than a friend, very fond of both daughters,_but particularly of Emma.  Between _them_ it was more the intimacy_of sisters.  Even before Miss Taylor had ceased to hold the nominal_office of governess, the mildness o

**The following questions refer to the entire text contained in this variable (which contains the text of multiple books).**

In [4]:
import pandas as pd
import numpy as np
import collections


## Part 1: Entropy and friends

### Questions:

**1. How many unique characters are used in the text?**


In [5]:
unique_chars = list(set(text))

print(f'The unique chars from text are:\n\n {unique_chars}\n')
print(f'{len(unique_chars)} unique characters are used in the text')

The unique chars from text are:

 ['v', 'a', '.', 'f', 'W', 'M', 'b', 'g', "'", 'B', 'J', 'R', '?', 'j', 'E', 'l', 'e', '8', 'N', 'x', 'q', '"', 'C', 'n', 'r', 'T', 'i', 'k', '1', '0', 'G', '9', 'F', 'p', 'm', 'D', 't', 'c', '7', 'U', 'I', 'K', 'Q', 'Z', 'X', 'o', '4', 'y', '6', 'O', '2', ' ', 'P', 'V', 'L', 'S', ',', 'z', '5', 'w', '!', 'A', 'u', 's', 'd', '3', '_', 'H', 'Y', 'h']

70 unique characters are used in the text


**2. Calculate the information content of characters: "a", "e", "x", " ".** 

$𝐼(𝑥_i)=−𝑙𝑜𝑔_2(𝑃(𝑥_i))$

Let's calculate char_count

In [6]:
d = collections.defaultdict(int)
for c in text:
    d[c] += 1
    
ch = pd.DataFrame(d.items(),columns=['chars','count_ch'])

And now we able to calculate probability of appearance character in the text and info content of needed characters


In [7]:
ch['proba'] = ch.count_ch/ch.count_ch.sum()

ch['info_content'] = -np.log2(ch.proba)

ch.loc[ch.chars.isin(["a", "e", "x", " "])]

,chars,count_ch,proba,info_content
3,a,698889,0.059261,4.076763
4,,2000723,0.169649,2.559376
9,e,1108892,0.094027,3.410779
42,x,8981,0.000762,10.358806


**Do these values look reasonable?** 

Yes, these values look reasonable

**Why ?**


Because from the dataframe ch we see, that the higher count in the text, the higher probability of appearance, the lower the information content of character we have.

**3. Calculate the character-level entropy and perplexity of the text.**


$𝐻=∑(𝑃(𝑥_𝑖)∗𝐼(𝑥_𝑖))$ <br>
$PP = 2^H$

In [8]:
shannon_entropy = np.sum(ch.proba * ch.info_content)
print(f'''character-level entropy of the text is  {shannon_entropy}

perplexity of the text is {2**shannon_entropy}''')

character-level entropy of the text is  4.498874036765774

perplexity of the text is 22.609764133453833


**4. What would be the character-level entropy and perplexity if characters were uniformly distributed?**

We have 70 chars in out text, so the probability of a character would be 1/70 and the entropy:
<br>$𝐻=−∑(𝑃(𝑥_i)∗𝑙𝑜𝑔_2(𝑃(𝑥_i)))$
Where $P = \frac{1}{len(ch)} = \frac{1}{70}  $ 

In [9]:
uniform_entropy = -len(ch)*(1/len(ch) * np.log2(1/len(ch)))

print(f'''
uniformly distributed character-level entropy of the text is  {uniform_entropy}
uniformly distributed perplexity of the text is {2**uniform_entropy}''')


uniformly distributed character-level entropy of the text is  6.129283016944966
uniformly distributed perplexity of the text is 70.0


 **Is this higher or lower than the values in 3, and why?**


It is higher than values in Q3, this is the maximum value for the entropy with such distribution

**5. What is the cross-entropy of the uniform distribution over characters relative to the sample distribution? <br>(H(P,Q) where P is the uniform distribution and Q is the sample distribution)**

$H(P,Q) = -\sum_{i}P(x_i) * log_2 Q(x_i) $

<br>
$P = \frac{1}{len(ch)} = \frac{1}{70}$<br>


$log_2 Q(x_i) =$  ch['inf_content']


$H(P,Q) =  - \frac{1}{70 } * \sum(log_2(Q(x_i))) = \frac{1}{70} * \sum(log_2(1/Q(x_i))) $


In [10]:
cross_entropy = (1/len(ch) * ch.info_content).sum()

print(f''' the cross-entropy of the uniform distribution over characters relative 
to the sample distribution is  {cross_entropy}''')

 the cross-entropy of the uniform distribution over characters relative 
to the sample distribution is  8.59586772830242



**6. Now consider a distribution where every character has probability 1/6 of being a space (" ") and 5/6 probability of being any other character (all with equal probability).** 

$𝐻(𝑃,𝑄)=𝑃((" ")) ∗ 𝑙𝑜𝑔_2(𝑄(" ")) + ∑(𝑃(𝑥_𝑖)∗𝑙𝑜𝑔_2(𝑄(𝑥_𝑖)))$
$= \frac{1}{6} ∗ 𝑙𝑜𝑔_2(𝑄("")) + ∑(\frac{5}{6} ∗ \frac{1}{(len(ch) -1)} ∗ 𝑙𝑜𝑔_2(𝑄(𝑥_𝑖)))$


x - other different characters, not `space`-character

**What is the cross-entropy of this distribution relative to the sample distribution?** 

In [11]:
#for all non-space-chars
ch['cross_entropy_5_6'] = ch.info_content * 5/6 * 1/(len(ch)-1)

#for `space`-chars
ch.loc[ch.chars == ' ', 'cross_entropy_5_6'] = 1/6 * ch.info_content 

ch.cross_entropy_5_6.sum()

7.66269031352059

**Is this higher or lower than the value from question 5, and why?**

The cross-entropy this time is lower than in Q5
<br>We were calculating it with different condition of probability 

**Not only are letters distributed unevenly in English, but they are also dependent on their neighbors. For example, the letter 'q' almost always appears before the letter 'u'. Let's examine some information-theoretic consequences of this:**

### Bonus question:
**7. (bonus) Calculate the mutual information of a character given its predecessor in the text.** 

Mutual Information is defined as follows:

$I(X,Y)= ∑P(x,y) * (log_2(\frac{P(x,y)}{P(x)*P(y)})$



**A *bigram* is a set of two adjacent characters - for example, the word "queen" contains bigrams "qu", "ue", "ee", and "en". We can get all the bigrams in our text as follows:**

In [12]:
bigrams_list = [text[i:i+2] for i in range(len(text)) if i < len(text) - 1]

Now, Let's Calculate the mutual information of a character given its predecessor in the text  <br>For this question for more convenient use in calculations  we will create a df here, It will help to follow the logic of calculations very clear

In [13]:
#count the bigrams
d = collections.defaultdict(int)
for c in bigrams_list:
    d[c] += 1

#create df
bi = pd.DataFrame(d.items(),columns=['bigram','count_bi'])

#probabilities into new df (P(x))
bi['proba_bi'] = bi.count_bi / bi.count_bi.sum()

#info_content into new df 
bi['info_content_bi'] = np.log2(1/bi.proba_bi)

#chars from bigrams into df
bi['c_1'] = bi.bigram.astype(str).str[0]
bi['c_2'] = bi.bigram.astype(str).str[1]

#add proba char 1
bi =bi.merge(ch[["proba","chars"]], left_on="c_1",right_on='chars', how="left")
bi.drop("chars", axis =1, inplace = True)
bi.rename(columns={"proba": "proba_c_1"}, inplace=True)

#add proba char 2
bi = bi.merge(ch[["proba","chars"]], left_on="c_2",right_on='chars', how="left")
bi.drop("chars", axis =1, inplace = True)
bi.rename(columns={"proba": "proba_c_2"}, inplace=True)

#counting mutual information by formula P(x,y) * (log(P(x,y)/P(x)*P(y))
bi['mutual_info']= bi.proba_bi * np.log2(bi.proba_bi / (bi.proba_c_1 * bi.proba_c_2))

bi

,bigram,count_bi,proba_bi,info_content_bi,c_1,c_2,proba_c_1,proba_c_2,mutual_info
0,_E,1740,1.475412e-04,12.726595,_,E,0.035187,0.000909,3.253284e-04
1,Em,991,8.403064e-05,13.538725,E,m,0.000909,0.018184,1.970717e-04
2,mm,5058,4.288870e-04,11.187115,m,m,0.018184,0.018184,1.609416e-04
3,ma,31552,2.675414e-03,8.546022,m,a,0.018184,0.059261,3.509941e-03
4,a,33356,2.828382e-03,8.465808,a,,0.059261,0.169649,-5.175001e-03
...,...,...,...,...,...,...,...,...,...
2063,",t",1,8.479379e-08,23.491466,",",t,0.016312,0.067873,-1.159343e-06
2064,",a",2,1.695876e-07,22.491466,",",a,0.016312,0.059261,-2.115901e-06
2065,KX,1,8.479379e-08,23.491466,K,X,0.000171,0.000015,4.261806e-07
2066,'6,1,8.479379e-08,23.491466,',6,0.001838,0.000556,-3.046169e-07


**What would this be if characters were all independent?**

If ${X }$ and ${ Y }$ are Independent then  $I(X:Y)= 0$ since ${P(x,y)}={P(x)*P(y)}$ There is no Mutual Information

### Questions:
**8. Calculate entropy and perplexity of the *bigrams* in the text.**


We defined all formulas for entropy and perplexity  in first questions so:

In [14]:
entropy_bigrams = np.sum(bi.proba_bi * bi.info_content_bi)
entropy_bigrams

8.036241352863215

In [15]:
perplexity_bigrams = 2**entropy_bigrams
perplexity_bigrams

262.5123257156022

**9. What would the entropy and perplexity be if all unique bigrams found in the text were uniformly distributed?**

From df `bi` we know that we have 2068 unique bigrams. We will do calculations the same way as we were doing in first questions about characters. $P=  \frac{1}{len(bi)} = \frac{1}{2068}  $ 

In [16]:
#H = -len(ch)*(1/len(ch) * np.log2(1/len(ch)))
entropy_uniform_unique_bigrams = -len(bi)*(1/len(bi)*np.log2(1/len(bi)))
entropy_uniform_unique_bigrams

11.014020470314934

And the  perplexity if all unique bigrams found in the text were uniformly distributed is:

In [17]:
perplexity_uniform_unique_bigrams = 2**entropy_uniform_unique_bigrams
perplexity_uniform_unique_bigrams

2067.999999999999

The same as in the first questions about characters, we got perplexity equals the number of unique bigrams

## Part 2: Coding theory

**ASCII (American Standard Code for Information Interchange) is a coding standard for representing basic characters as a sequence of seven binary digits. For example, in ASCII the character "a" is represented as 1100001.**

**Since the number of characters needed in everyday use of computers has expanded greatly since ASCII was released in 1963, text is usually encoded with more bits (e.g. in UTF-8 encoding). ASCII cannot represent characters in languages with non-Latin character sets or various other special characters.**

### Questions:

**10. What is the ASCII binary representation of the following string: "HelloWorld"? Use the Python functions bin() ord() to find the binary representation - do not do this manually.**

In [18]:
def string2bits(s=''):
    return ''.join([bin(ord(x))[2:].zfill(8) for x in s])


s =  "HelloWorld"
b = string2bits(s)
print(b)

01001000011001010110110001101100011011110101011101101111011100100110110001100100



**11. What is the number of bits required in ASCII total and per character to represent our text?**


ASCII character set contains a total of 128 characters and each character has a unique value between 0 and 127. Hence a 7-bit binary number is sufficient to represent a character from ASCII charset since a 7-bit binary number can hold values from 0 to 127 (total of 128 unique values → ²⁷).

The bit-width or bit-length is the length of the binary number used by an encoding scheme to represent a character. Hence in ASCII, the bit-width is 7.

However, in a typical computer system, the memory is made of unit cells and each individual cell contains 8 bits (byte). Hence, even though ASCII needs only 7 bits to encode a character, it is stored as 8 bit by keeping first bit 0 (MSB). Hence, the actual bit-width of ASCII is 8.


Since the first bit of ASCII character is always 0, it is also called as dead bit as it has no significance or use.

In [19]:
len(string2bits(text))#for 8 bites

94346544

In [20]:
len(text)*7 #for 7 bites

82553226

**12. Is it possible to come up with a better fixed-length binary code for single characters in our text?** 

No. Because of Fixed-length code  takes ⌈log 128⌉ or 7 bits to code the 128 symbols

**How about a better variable-length binary code? (You don't need to construct such codes, just state whether they exist and why.)**

Yes, we can use Huffman's algorithm:
The output from Huffman's algorithm can be viewed as a variable-length code table for encoding a source symbol (such as a character in a file). The algorithm derives this table from the estimated probability or frequency of occurrence (weight) for each possible value of the source symbol. As in other entropy encoding methods, more common symbols are generally represented using fewer bits than less common symbols. Huffman's method can be efficiently implemented, finding a code in time linear to the number of input weights if these weights are sorted. However, although optimal among methods encoding symbols separately, Huffman coding is not always optimal among all compression methods - it is replaced with arithmetic coding or asymmetric numeral systems if better compression ratio is required.


**13. (bonus) What does the source-coding theorem predict should be approximately possible for our (long) text?**

$N$ i.i.d. random variables each with entropy $H(X)$ can be compressed into more than $N H(X)$ bits with negligible risk of information loss, as $N → ∞$; but conversely, if they are compressed into fewer than $N H(X)$ bits it is virtually certain that information will be lost.


**14. (bonus) For the letters {"a", "b", "c", "d"}, why is the variable-length binary code {"a": "011", "b": "0110", "c": "1110", "d": "10011"} not uniquely decodeable? Give an example. <BR>Hint: calculate encodings of all possible words up to length 3 .**

Consider the variable length code  {"a": "011", "b": "0110", "c": "1110", "d": "10011"} . A segment of encoded message such as ‘01101110011’ can be decoded in more than one way. For example, ‘01101110011’ can be interpreted in at least two ways, as $'aad'$ or as $'bca'$.




The code is not uniquely decodable and therefore cannot be used for data compression.